# Código de Mask R-CNN modificado por Daniel Cay



The MIT License (MIT)

Copyright (c) 2017 Matterport, Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.



In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf
from numpy import asarray
from numpy import savetxt

# paquetes para el tracker
import imutils
import time
import cv2


# Root directory of the project
ROOT_DIR = os.path.abspath("")
print("ROOT_DIR->",ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR+"/codigo")  # To find local version of the library
from funcionesAux import utils, visualize, metricas, obtenerMascaras, io, training, asociarObjetos, algPrediccion
import funcionesAux.model as modellib

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "/codigo/coco/"))  # To find local version
from coco import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR+"/codigo", "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directorio de los videos a procesar
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
print("VIDEO_DIR->",VIDEO_DIR)

#Directorio de los videos procesados
VIDEO_PROCESADO_DIR = os.path.join(ROOT_DIR, "videosProcesados")


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## CNN con descriptores aprendidos

In [ ]:
import keras 
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Input, concatenate, Activation, Subtract

from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.losses import binary_crossentropy


#(14,14,256)->(1,1,256)
def bloqueReduccion(in1,in2):
    x1=Conv2D(256, (5, 5), strides=(2,2))
    x2=BatchNormalization()
    x3=Activation('relu')
    x4=Conv2D(256, (5, 5))
    x5=BatchNormalization()
    x6=Activation('relu')
    
    #objeto 1
    ob1=x1(in1)
    ob1=x2(ob1)
    ob1=x3(ob1)
    ob1=x4(ob1)
    ob1=x5(ob1)
    ob1=x6(ob1)
    
    #objeto 2
    ob2=x1(in2)
    ob2=x2(ob2)
    ob2=x3(ob2)
    ob2=x4(ob2)
    ob2=x5(ob2)
    ob2=x6(ob2)
    
    return ob1,ob2

def createModel():

    in1 = Input(shape=(14,14,256))
    in2 = Input(shape=(14,14,256))

    #(1,1,256)->(256)
    reduccion_in1,reduccion_in2 = bloqueReduccion(in1,in2)
    reduccion_in1 = Flatten()(reduccion_in1)
    reduccion_in2 = Flatten()(reduccion_in2)

    #(reduccion_in1-reduccion_in2)
    z = Subtract()([reduccion_in1, reduccion_in2])

    z=Dense(256, activation='relu')(z)
    z=Dense(128, activation='relu')(z)
    z=Dense(64, activation='relu')(z)
    z=Dense(1, activation='sigmoid')(z)

    modelRAM = Model(inputs=[in1,in2], outputs=[z])

    modelRAM.compile(Adam(lr=.0001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return modelRAM




## Seguimiento y Segmentación de Múltiples Objetos con Descriptores Aprendidos

In [ ]:
def trackingVideo(video,pesoMask,pesoRAM,umbral):
    primeraVez=True
    primerFrameProcesado=True
        
    #lista de objetos vistos entre todos los frames hasta el actual
    objetos=[]
    #lista de los objetos detectados en el frame actual
    objetosFrame=[]
    #identificador único de un objeto (ultimos ID usado)
    ultimoID=0
    #generar colores para cada ID
    colores=visualize.random_colors(1000)
    #generar colores para cada ID del groundtruth
    coloresGt=visualize.random_colors(1000)
    
    #CARGAR CNN con descriptores aprendidos en la época 136
    modelRAM = createModel()
    modelRAM.load_weights("codigo/cnn/cp-0136.ckpt")
    
    #lista para formar el video procesado a partir del procesamiento de cada frame
    img_array = []
    
    #Leer video para su procesamiento
    cap=cv2.VideoCapture(VIDEO_DIR+'/'+video)
    i=0
    #Mientras haya frames que leer del video...
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        
        #OpenCV lee las imagenes en BGR, asi que hay que convertirla a RGB
        frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        #Instance segmentation del frame actual
        results = model.detect([frame], primeraVez, colores, verbose=0) 
        r = results[0]
        
        #Si es el primer frame en el que se detectan objetos...
        if primeraVez:
            objetosFrame=r
            objetos=r
            ultimoID+=len(r['rois'])
            if len(objetosFrame['ids'])>0:
                primeraVez=False
            
        #El resto de frames ya se pueden comparar con los anteriores...
        else:  
            objetos=algPrediccion.actualizarTrackers(objetos,frame)
            
            objetos,objetosFrame,ultimoID=asociarObjetos.objetosMostrar(modelRAM,objetos,r,ultimoID,colores,pesoMask,pesoRAM)
            
            objetos=algPrediccion.iniciarTrackers(objetos,frame)
            
            objetos=asociarObjetos.borrarObjetosDesfasados(objetos,10)
            
        #Guardar cada frame procesado individualmente
        sep = '.'
        videoSinExtension = video.split(sep, 1)[0]
        carpetaVideo = os.path.join(VIDEO_PROCESADO_DIR, videoSinExtension)
        ruta='videosProcesados/'+videoSinExtension+'/'+str(i)+'.png'
        if primerFrameProcesado:
            os.mkdir(carpetaVideo)
            primerFrameProcesado=False
        
        #Mostrar frame procesado por pantalla
        visualize.display_instances(frame, ruta, objetosFrame['ids'], objetosFrame['colors'], objetosFrame['rois'], 
                                objetosFrame['masks'], objetosFrame['class_ids'], class_names, objetosFrame['scores'])
        
        #Leer frame procesado y añadirlo a la lista de frames procesados para formar un video
        frameProcesado = cv2.imread('videosProcesados/'+videoSinExtension+'/'+str(i)+'.png')
        height, width, layers = frameProcesado.shape
        size = (width,height)
        img_array.append(frameProcesado)

        i+=1
        
    cap.release()
    
    #Crear el video procesado uniendo todos los frames procesados
    out = cv2.VideoWriter(VIDEO_PROCESADO_DIR+'/'+video,cv2.VideoWriter_fourcc(*'DIVX'), 10.0, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()



## Ajustar parámetros del sistema:
    - Peso dado a la información del solapamiento
    - Peso dado a la información de la CNN con descriptores aprendidos
    - Umbral de asociación con un objeto del groundtruth

In [ ]:
def procesarVideo(video):
    pesoMask=0.8
    pesoRAM=0.2
    umbral=0.5

    trackingVideo(video, pesoMask, pesoRAM, umbral)
    
    print("-------------------------")
    print("VIDEO PROCESADO CON ÉXITO")
    print("-------------------------")

# UTILIZAR ESTA FUNCIÓN 
    Ejemplo de uso:
    ---------------
        1) TFG/videos->prueba.mp4                    (colocar vídeo a procesar en la carpeta "videos")
            2) Ejecutar -> procesarVideo(prueba.mp4)     (no olvidar la extension del video)
        Se genera en la carpeta "videosProcesados":
            2) TFG/videosProcesados/prueba.mp4       (el video procesado)
            3) TFG/videosProcesados/prueba           (una carpeta con todos los frames procesados individualmente)

In [ ]:
procesarVideo('prueba.mp4')